In [1]:
import os
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy import stats
from functools import reduce
from patsy import dmatrices
import statsmodels.formula.api as sm
import statsmodels.formula.api as smf

In [2]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/descriptive/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'cits.csv'),
                  sep=",", low_memory=False)
data = data[['distname', 'treat', 'year', 'students_hisp', 'type_rural', 'type_suburban', 'type_urban', 'students_num']]

In [3]:
df = data = data[['distname', 'treat', 'year', 'students_hisp', 'type_rural', 'type_suburban', 'type_urban', 'students_num']]

df.reset_index().head()

,index,distname,treat,year,students_hisp,type_rural,type_suburban,type_urban,students_num
0,0,CAYUGA ISD,0,2012,0.050847,1,0,0,590
1,1,ELKHART ISD,0,2012,0.078678,0,0,0,1271
2,2,FRANKSTON ISD,0,2012,0.103683,1,0,0,733
3,3,NECHES ISD,0,2012,0.099256,1,0,0,403
4,4,PALESTINE ISD,0,2012,0.389752,0,0,0,3220


In [4]:
df_dist = df[df.year == 2015][['distname', 'treat', 'type_rural', 'type_suburban', 'type_urban']]
df_12 = df[df.year == 2012][['distname', 'students_hisp', 'students_num']].rename(columns = {'students_hisp': 'hisp12', 'students_num': 'num12'})
df_13 = df[df.year == 2013][['distname', 'students_hisp', 'students_num']].rename(columns = {'students_hisp': 'hisp13', 'students_num': 'num13'})
df_14 = df[df.year == 2014][['distname', 'students_hisp', 'students_num']].rename(columns = {'students_hisp': 'hisp14', 'students_num': 'num14'})
df_15 = df[df.year == 2015][['distname', 'students_hisp', 'students_num']].rename(columns = {'students_hisp': 'hisp15', 'students_num': 'num15'})
df = reduce(lambda x, y: pd.merge(x, y, on = 'distname'), [df_dist, df_12, df_13, df_14, df_15])
df.tail()

,distname,treat,type_rural,type_suburban,type_urban,hisp12,num12,hisp13,num13,hisp14,num14,hisp15,num15
865,GRAHAM ISD,0,0,0,0,0.254257,2525,0.264180,2574,0.286547,2572,0.304731,2494
866,NEWCASTLE ISD,0,1,0,0,0.095745,188,0.128205,195,0.132701,211,0.100000,210
867,OLNEY ISD,0,1,0,0,0.369681,752,0.372654,746,0.362117,718,0.365517,725
868,ZAPATA COUNTY ISD,0,0,1,0,0.990449,3455,0.988842,3585,0.991101,3596,0.989937,3677
869,LA PRYOR ISD,0,1,0,0,0.944325,467,0.941788,481,0.928571,476,0.939959,483


In [5]:
y, X = dmatrices('treat ~ type_rural + type_suburban + type_urban + hisp12 + hisp13 + hisp14 + hisp15 + num12 + num13 + num14 + num15',
                 data=df, return_type='dataframe', NA_action = "drop")


In [6]:
mod = sm.Logit(y, X)    # Describe model
res = mod.fit()       # Fit model
print(res.summary())   # Summarize model

PerfectSeparationError: Perfect separation detected, results not available

In [ ]:
df['yhat'] = res.predict(X)
df[df.treat == 0].tail()

In [ ]:
plt.style.use('seaborn')
my_dpi=96
plt.figure(figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)


kwargs = dict(histtype='stepfilled', alpha=0.3, bins=40)
plt.hist(df[df.treat == 0].yhat, **kwargs, label = "Eligible TPSDs")
plt.hist(df[df.treat == 1].yhat, **kwargs, label = "DOIs")

plt.legend()


plt.xlabel('Propensity Score')
plt.title('Overlap')
plt.ylabel('Count')
plt.show()


# Weight

In [ ]:
df['weight'] = df.treat+(1-df.treat)*df.yhat/(1-df.yhat)
df.head()

## Trim
No one observation weighted more than 4%

In [ ]:
total_weight = sum(df[df.treat == 0].weight)*len(df[df.treat == 0])
df['wtshare'] = (df.weight/total_weight)*100
df[df.wtshare > 4]

# Merge PS Weight

In [ ]:
cits_match = pd.merge(data, df, on = 'distname')
cits_match.to_csv(os.path.join(data_path, 'clean', 'cits_match.csv'), sep=",")
cits_match.head()